In [1]:
import os
import cdflib
import xarray as xr
import numpy as np

# Define the input CDF file path and output NetCDF file path
cdf_file = r"C:\Users\parva\OneDrive\Desktop\Inspire_Project\Geo_B_Storm\TIMED_Pressure_Data\timed_l2as_saber_20211028000052_20211111235913_cdaweb.cdf"
netcdf_file = r"C:\Users\parva\OneDrive\Desktop\Inspire_Project\Geo_B_Storm\TIMED_Pressure_Data\timed_l2as_saber_20211028000052_20211111235913_cdaweb.nc"

# Check if the CDF file exists
if not os.path.exists(cdf_file):
    raise FileNotFoundError(f"The file {cdf_file} does not exist.")

# Load the CDF file using cdflib
cdf_data = cdflib.CDF(cdf_file)

# Get variable names and global attributes from the CDF file
variables = cdf_data.cdf_info().zVariables
global_attrs = cdf_data.globalattsget()

# Create an xarray Dataset
ds = xr.Dataset()

for var in variables:
    # Get the variable data and attributes
    data = cdf_data.varget(var)
    attrs = cdf_data.varattsget(var)
    dims = attrs.get('DEPEND_0', ['dim_' + var])  # Use dependency if available, or fallback

    # Handle single-dimension cases
    if isinstance(dims, str):
        dims = [dims]

    # Rename dimension if it conflicts with the variable name
    if var in dims:
        dims = [dim if dim != var else f"{var}_dim" for dim in dims]

    # Adjust dimensions for 2D variables
    if len(dims) != data.ndim:
        if len(data.shape) == 2:
            dims = ['Epoch', f'{var}_dim']
            ds[dims[1]] = (dims[1], np.arange(data.shape[1]))
        else:
            print(f"Skipping variable {var}: dims={dims}, shape={data.shape}")
            continue

    # Handle 'Epoch' as a special case
    if var == 'Epoch':
        ds.coords[var] = (dims, data)
    else:
        # Create variable in the xarray Dataset
        ds[var] = (dims, data)
        # Assign variable attributes
        for key, value in attrs.items():
            ds[var].attrs[key] = value

# Add global attributes
for key, value in global_attrs.items():
    ds.attrs[key] = value

# Save the Dataset to NetCDF format
ds.to_netcdf(netcdf_file)
print(f"NetCDF file saved at {netcdf_file}")

NetCDF file saved at C:\Users\parva\OneDrive\Desktop\Inspire_Project\Geo_B_Storm\TIMED_Pressure_Data\timed_l2as_saber_20211028000052_20211111235913_cdaweb.nc


In [2]:
import xarray as xr

# Path to the NetCDF file
netcdf_file = "C:\\Users\\parva\\OneDrive\\Desktop\\Inspire_Project\\Geo_B_Storm\\TIMED_Pressure_Data\\timed_l2as_saber_20211028000052_20211111235913_cdaweb.nc"

# Open the NetCDF file
ds = xr.open_dataset(netcdf_file)

In [3]:
ds

<xarray.Dataset>
Dimensions:           (dim_Epoch: 21395, Epoch: 21395, tpSolarLT_dim: 400,
                       tpgpaltitude_dim: 400, pressure_dim: 400,
                       ktemp_dim: 400, density_dim: 400, tpaltitude_dim: 400)
Coordinates:
    Epoch             (dim_Epoch) float64 ...
  * tpSolarLT_dim     (tpSolarLT_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
  * tpgpaltitude_dim  (tpgpaltitude_dim) int32 0 1 2 3 4 ... 395 396 397 398 399
  * pressure_dim      (pressure_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
  * ktemp_dim         (ktemp_dim) int32 0 1 2 3 4 5 ... 394 395 396 397 398 399
  * density_dim       (density_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
  * tpaltitude_dim    (tpaltitude_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
Dimensions without coordinates: dim_Epoch
Data variables:
    tplatitudeAVG     (Epoch) float32 ...
    tplongitudeAVG    (Epoch) float32 ...
    tpSolarLT         (Epoch, tpSolarLT_dim) float32 ...
    tpgpaltitude      (Epoch, tpgpaltitude_dim) float32 ...
    pressure          (Epoch, pressure_dim) float32 ...
    ktemp             (Epoch, ktemp_dim) float32 ...
    density           (Epoch, density_dim) float64 ...
    tpaltitude        (Epoch, tpaltitude_dim) float32 ...
Attributes: (12/26)
    Project:                     TIMED
    Source_name:                 TIMED>Thermosphere+Ionosphere+Mesosphere+Ene...
    Discipline:                  Space Physics>Ionospheric Science
    Data_type:                   L2A>Level 2A version
    Descriptor:                  SABER>Sounding of the Atmosphere using Broad...
    Data_version:                1
    ...                          ...
    LINK_TITLE:                  ['SABER data and experiment home page at NAS...
    HTTP_LINK:                   ['http://saber.gats-inc.com/', 'http://www.t...
    Mission_group:               ['TIMED', '!___ITM Data including Earth Imag...
    spase_DatasetResourceID:     spase://NASA/NumericalData/TIMED/SABER/L2A/V...
    CDAWEB_PARENTS:              ['timed_l2a_saber_00000000_v01', 'timed_l2a_...
    PARENTS:                     ['timed_l2a_saber_00000000_v01', 'timed_l2a_...

In [4]:
print(ds.variables)

Frozen({'Epoch': <xarray.Variable (dim_Epoch: 21395)>
[21395 values with dtype=float64], 'tplatitudeAVG': <xarray.Variable (Epoch: 21395)>
[21395 values with dtype=float32]
Attributes:
    FIELDNAM:   Tangent Point latitude averaged
    CATDESC:    Tangent point latitude (average ) over lower part of scan
    DEPEND_0:   Epoch
    FILLVAL:    -999.0
    FORMAT:     f10.1
    LABLAXIS:   Lati TPoint bot
    UNITS:      degree
    VALIDMIN:   -90.0
    VALIDMAX:   90.0
    VAR_TYPE:   data
    DIM_SIZES:  0, 'tplongitudeAVG': <xarray.Variable (Epoch: 21395)>
[21395 values with dtype=float32]
Attributes:
    FIELDNAM:   Tangent Point longitude averaged
    CATDESC:    ---------> longitude (lower average)
    DEPEND_0:   Epoch
    FILLVAL:    -999.0
    FORMAT:     f10.1
    LABLAXIS:   Longi TPoint bot
    UNITS:      degree
    VALIDMIN:   0.0
    VALIDMAX:   360.0
    VAR_TYPE:   data
    DIM_SIZES:  0, 'tpSolarLT_dim': <xarray.IndexVariable 'tpSolarLT_dim' (tpSolarLT_dim: 400)>
array([

In [5]:
cleaned_data = ds.where(ds != -999.0, drop=False)

In [6]:
ds

<xarray.Dataset>
Dimensions:           (dim_Epoch: 21395, Epoch: 21395, tpSolarLT_dim: 400,
                       tpgpaltitude_dim: 400, pressure_dim: 400,
                       ktemp_dim: 400, density_dim: 400, tpaltitude_dim: 400)
Coordinates:
    Epoch             (dim_Epoch) float64 ...
  * tpSolarLT_dim     (tpSolarLT_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
  * tpgpaltitude_dim  (tpgpaltitude_dim) int32 0 1 2 3 4 ... 395 396 397 398 399
  * pressure_dim      (pressure_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
  * ktemp_dim         (ktemp_dim) int32 0 1 2 3 4 5 ... 394 395 396 397 398 399
  * density_dim       (density_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
  * tpaltitude_dim    (tpaltitude_dim) int32 0 1 2 3 4 5 ... 395 396 397 398 399
Dimensions without coordinates: dim_Epoch
Data variables:
    tplatitudeAVG     (Epoch) float32 -24.55 -28.63 -31.15 ... -68.54 -72.38
    tplongitudeAVG    (Epoch) float32 147.8 148.0 148.1 ... 101.5 99.78 95.81
    tpSolarLT         (Epoch, tpSolarLT_dim) float32 9.983 9.982 ... -999.0
    tpgpaltitude      (Epoch, tpgpaltitude_dim) float32 151.1 150.8 ... -999.0
    pressure          (Epoch, pressure_dim) float32 -999.0 -999.0 ... -999.0
    ktemp             (Epoch, ktemp_dim) float32 -999.0 -999.0 ... -999.0 -999.0
    density           (Epoch, density_dim) float64 -999.0 -999.0 ... -999.0
    tpaltitude        (Epoch, tpaltitude_dim) float32 155.0 154.6 ... -999.0
Attributes: (12/26)
    Project:                     TIMED
    Source_name:                 TIMED>Thermosphere+Ionosphere+Mesosphere+Ene...
    Discipline:                  Space Physics>Ionospheric Science
    Data_type:                   L2A>Level 2A version
    Descriptor:                  SABER>Sounding of the Atmosphere using Broad...
    Data_version:                1
    ...                          ...
    LINK_TITLE:                  ['SABER data and experiment home page at NAS...
    HTTP_LINK:                   ['http://saber.gats-inc.com/', 'http://www.t...
    Mission_group:               ['TIMED', '!___ITM Data including Earth Imag...
    spase_DatasetResourceID:     spase://NASA/NumericalData/TIMED/SABER/L2A/V...
    CDAWEB_PARENTS:              ['timed_l2a_saber_00000000_v01', 'timed_l2a_...
    PARENTS:                     ['timed_l2a_saber_00000000_v01', 'timed_l2a_...